In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# copy a test parquet
import os
import shutil

os.environ["DSML_CURR_RUN_ID"]="run-00-01-01-000000"
current_runid = os.getenv("DSML_CURR_RUN_ID")

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
from sinara.substep import NotebookSubstep, get_user, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    
    tmp_outputs = [
        { ENTITY_NAME: "testParquet" },
        { ENTITY_NAME: "entityM" },
        { ENTITY_NAME: "entityN" }
    ],
   
    custom_outputs = 
    [
        { ENTITY_NAME: "entityA", ENTITY_PATH: f"/data/home/{get_user()}/pipeline/zone/prev_step/{current_runid}/entityA" },
        { ENTITY_NAME: "entityB", ENTITY_PATH: f"/data/home/{get_user()}/pipeline/zone/prev_step/{current_runid}/entityB" },
        { ENTITY_NAME: "entityC", ENTITY_PATH: f"/data/home/{get_user()}/pipeline/zone/prev_step/{current_runid}/entityC" },
        { ENTITY_NAME: "entityI", ENTITY_PATH: f"/data/preprod/other_pipeline/monthly_run/ext_step/{current_runid}/entityI" },
        { ENTITY_NAME: "entityJ", ENTITY_PATH: f"/data/home/{get_user()}/other_pipeline2/zone0/ext_step2/{current_runid}/entityJ" },
    ]
)

substep.print_interface_info()

In [ ]:
# create tmp outputs for the substep_full
tmp_parquet_path = substep.tmp_outputs().testParquet
entityM_path = substep.tmp_outputs().entityM

!wget https://github.com/apache/parquet-testing/raw/master/data/alltypes_dictionary.parquet -O {tmp_parquet_path}/test.parquet
!cp {tmp_parquet_path}/test.parquet {entityM_path}
!cp {tmp_parquet_path}/test.parquet {substep.tmp_outputs().entityN}

In [ ]:
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
# write entities for substep_full
df = spark.read.parquet(tmp_parquet_path)

outputs = substep.custom_outputs()
df.write.parquet(outputs.entityA, mode='overwrite')
df.write.parquet(outputs.entityB, mode='overwrite')
df.write.parquet(outputs.entityC, mode='overwrite')
df.write.parquet(outputs.entityI, mode='overwrite')
df.write.parquet(outputs.entityJ, mode='overwrite')

In [ ]:
SinaraSpark.stop_session()